In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import requests
import math
headers = {'x-api-key': '2PgPyJCL7hxItnAfYnNIwvvHlQvITosKzweDA04K', 'Content-Type': 'application/json'}

from nltk.tokenize import word_tokenize
from nltk.tag import CRFTagger

# Pre-processing

In [2]:
def normalisasi(corpus):
    hasil_normalisasi = []
    for tweet in corpus['tweet']:
        #normalisasi: menghapus huruf berulang
        tweet2 = word_tokenize(tweet)
        last_char = [word[-1] for word in tweet2]
        result = []
        for index, item in enumerate(last_char):
            hasil = re.sub(r''+re.escape(item)+r'{2,}$', re.escape(item), tweet2[index])
            result.append(hasil)
        tweet = " ".join(result)
                
        #request normalisasi dengan prosa
        body = {'text': tweet}
        r_pos = requests.post('https://api.prosa.ai/v1/normals', headers = headers, json = body)
        tweet = r_pos.json()['text']
        
        hasil_normalisasi.append(tweet)
        
    corpus['normalisasi'] = hasil_normalisasi
    return corpus

In [3]:
def preprocessing(corpus):
    cleantweet = []
    tokentweet = []
    for tweet in corpus['normalisasi']:
        
        #huruf kecil
        tweet = tweet.lower()
        
        #datacleaning (link)
        tweet = re.sub(r'[a-zA-Z0-9]{0,}\.[c][o][m][^\s]{0,}', ' ', tweet)
        
        #datacleaning (username dan hashtag)
        tweet = re.sub(r'@\S*\s?|#\S*\s?', ' ', tweet) 
        
        #datacleaning (numbers)
        tweet = re.sub(r'[0-9]', ' ', tweet)
        
        #datacleaning (special character)
        tweet = re.sub(r'[\!\"\”\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\[\\\]\^\_\`\{\|\}\~]', ' ', tweet)
        
        #tokenisasi
        tweet = word_tokenize(tweet)
        
        #removing kata 'nya' yang terpisah dari kata hasil normalisasi
        #stopword = 'nya'
        
        token = [word for word in tweet]
        
        tokentweet.append(token)
        cleantweet.append(" ".join(token))
    
    corpus['clean'] = cleantweet
    corpus['token'] = tokentweet
    return corpus

# Term Frequency

In [4]:
def term_frequency(corpus):
    token = []
    for item in corpus['token']:
        for item2 in item:
            if item2 not in token:
                token.append(item2)
    token.sort()
    
    doc_tf = pd.DataFrame(token, columns=['token'])
    
    for index, item in enumerate(corpus['token']):
        tf_inTweet = []
        for index2, item2 in enumerate(token):
            frequency = item.count(item2)
            tf_inTweet.append(frequency)
        doc_tf[str(index+1)] = tf_inTweet
    
    doc_tf.set_index('token', inplace=True)
    
    return doc_tf

# Read Data Latih

In [5]:
#2 data latih
data_latih = pd.read_csv("data latih.csv", sep=",")
data_latih1 = pd.read_csv("data latih1.csv", sep=",")
data_latih2 = pd.read_csv("data latih2.csv", sep=",")
data_latih3 = pd.read_csv("data latih3.csv", sep=",")
data_latih4 = pd.read_csv("data latih4.csv", sep=",")
data_latih5 = pd.read_csv("data latih5.csv", sep=",")
data_latih6 = pd.read_csv("data latih6.csv", sep=",")
data_latih7 = pd.read_csv("data latih7.csv", sep=",")
data_latih8 = pd.read_csv("data latih8.csv", sep=",")

# Normalisasi Data Latih

In [6]:
#norm_latih1 = normalisasi(data_latih1.copy())

In [7]:
#norm_latih2 = normalisasi(data_latih2.copy())

In [8]:
#norm_latih3 = normalisasi(data_latih3.copy())

In [9]:
#norm_latih4 = normalisasi(data_latih4.copy())

In [10]:
#norm_latih5 = normalisasi(data_latih5.copy())

In [11]:
#norm_latih6 = normalisasi(data_latih6.copy())

In [12]:
#norm_latih7 = normalisasi(data_latih7.copy())

In [13]:
#norm_latih8 = normalisasi(data_latih8.copy())

In [14]:
#norm_latih = pd.concat([norm_latih1, norm_latih2, norm_latih3, norm_latih4, norm_latih5, norm_latih6, norm_latih7, norm_latih8], ignore_index=True)
#norm_latih.to_csv("Hasil Normalisasi Data Latih.csv", index=False)

# Pre-processing Data Latih

In [15]:
data_training = pd.read_csv("Hasil Normalisasi Data Latih.csv")

In [16]:
cleanLatih = preprocessing(data_training)
#cleanLatih.to_csv("Hasil Preprocessing Data Latih.csv")
cleanLatih

,tweet,kelas,normalisasi,clean,token
0,"MRT JAKARTA . . @mrtjakarta, ga macet, bersih,...",positif,"MRT JAKARTA..@mertjakarta, enggak macet, bersi...",mrt jakarta enggak macet bersih dingin dan nya...,"[mrt, jakarta, enggak, macet, bersih, dingin, ..."
1,Andalan baruku buat pulang kerumah. Semoga tah...,positif,Andalan baru ku buat pulang ke rumah. Semoga t...,andalan baru ku buat pulang ke rumah semoga ta...,"[andalan, baru, ku, buat, pulang, ke, rumah, s..."
2,sekarang gw gak takut lagi keabisan tije kalo ...,positif,sekarang gue enggak takut lagi kehabisan tipe ...,sekarang gue enggak takut lagi kehabisan tipe ...,"[sekarang, gue, enggak, takut, lagi, kehabisan..."
3,masih jaman naik motor gunain dong transportas...,positif,masih zaman naik motor gunakan dong transporta...,masih zaman naik motor gunakan dong transporta...,"[masih, zaman, naik, motor, gunakan, dong, tra..."
4,Thankyou @mrtjakarta maen jauh jdi deket,positif,thankyou@mertjakarta main jauh jadi dekat,thankyou main jauh jadi dekat,"[thankyou, main, jauh, jadi, dekat]"
5,Thanks @mrtjakarta untuk perjalanannya di rama...,positif,terima kasih@mertjakarta untuk perjalanan nya ...,terima kasih untuk perjalanan nya di ramadhan ini,"[terima, kasih, untuk, perjalanan, nya, di, ra..."
6,Suassna MRT sore ini.... LANCAR!! @mrtjakarta,positif,suasana MRT sore ini\.. LANCAR!!@mertjakarta,suasana mrt sore ini lancar,"[suasana, mrt, sore, ini, lancar]"
7,Can't wait.. Smga pembangunannya lancar sampai...,positif,ca dan ' t wait\. semoga pembangunan nya lanca...,ca dan t wait semoga pembangunan nya lancar sa...,"[ca, dan, t, wait, semoga, pembangunan, nya, l..."
8,Akhirnya Headway @mrtjakarta sekarang sudah me...,positif,Akhirnya Headway@mertjakarta sekarang sudah me...,akhirnya headway sekarang sudah meningkat ke m...,"[akhirnya, headway, sekarang, sudah, meningkat..."
9,Terima kasih @mrtjakarta saya bisa buka puasa ...,positif,Terima kasih@mertjakarta saya bisa buka puasa ...,terima kasih saya bisa buka puasa di rumah di ...,"[terima, kasih, saya, bisa, buka, puasa, di, r..."


# Read Data Uji

In [17]:
#2 data uji
data_uji = pd.read_csv("data uji.csv", sep=",")
data_uji1 = pd.read_csv("data uji1.csv", sep=",")
data_uji2 = pd.read_csv("data uji2.csv", sep=",")

# Normalisasi Data Uji

In [18]:
#norm_uji1 = normalisasi(data_uji1.copy())

In [19]:
#norm_uji2 = normalisasi(data_uji2.copy())

In [20]:
#norm_uji = pd.concat([norm_uji1, norm_uji2], ignore_index=True)
#norm_uji.to_csv("Hasil Normalisasi Data Uji.csv", index=False)

# Pre-processing Data Uji

In [21]:
data_testing = pd.read_csv("Hasil Normalisasi Data Uji.csv")

In [22]:
cleanUji = preprocessing(data_testing)
#cleanUji.to_csv("Hasil Preprocessing Data Uji.csv")
cleanUji

,tweet,kelas aktual,normalisasi,clean,token
0,Senangnya toilet stasiun MRT masih bersih! (@...,positif,Senang nya toilet stasiun MRT masih bersih!(@S...,senang nya toilet stasiun mrt masih bersih mrt...,"[senang, nya, toilet, stasiun, mrt, masih, ber..."
1,Biar ada pengalaman jalan2 k stasiun Lebak Bul...,positif,Biar ada pengalaman jalan-jalan ke stasiun Leb...,biar ada pengalaman jalan jalan ke stasiun leb...,"[biar, ada, pengalaman, jalan, jalan, ke, stas..."
2,Cantik nih stasiun MRT nya @mrtjakarta,positif,Cantik ini stasiun MRT nya@mertjakarta,cantik ini stasiun mrt nya,"[cantik, ini, stasiun, mrt, nya]"
3,"Semoga rutenya cepat diperluas ya, nyaman bgt ...",positif,"Semoga rute nya cepat diperluas ya, nyaman ban...",semoga rute nya cepat diperluas ya nyaman banget,"[semoga, rute, nya, cepat, diperluas, ya, nyam..."
4,Hari ini pertama kali naik @mrtjakarta. Akhirn...,positif,Hari ini pertama kali naik@mertjakarta. Akhirn...,hari ini pertama kali naik akhirnya jakarta pu...,"[hari, ini, pertama, kali, naik, akhirnya, jak..."
5,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,positif,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,di stasiun mrt dukuh atas kondisi bersih dan n...,"[di, stasiun, mrt, dukuh, atas, kondisi, bersi..."
6,Mewah dan Modern. 2 kata itu sepertinya cukup ...,positif,Mewah dan Modern. 2 kata itu sepertinya cukup ...,mewah dan modern kata itu sepertinya cukup men...,"[mewah, dan, modern, kata, itu, sepertinya, cu..."
7,Salut dan makasi kpd mas penjaga di Stasiun Bl...,positif,Salut dan terima kasih kepada mas penjaga di S...,salut dan terima kasih kepada mas penjaga di s...,"[salut, dan, terima, kasih, kepada, mas, penja..."
8,"Mantap mrt jakarta!!! Keren, kalian sigap!",positif,"Mantap mrt jakarta!!! Keren, kalian sigap!",mantap mrt jakarta keren kalian sigap,"[mantap, mrt, jakarta, keren, kalian, sigap]"
9,gilak nih dingin nya gerbong.... top dah @mrtj...,positif,gila ini dingin nya gerbong\.. top sudah@mertj...,gila ini dingin nya gerbong top sudah,"[gila, ini, dingin, nya, gerbong, top, sudah]"


# Term Frequency Data Latih

In [23]:
#1 TF all data
raw_tf = term_frequency(cleanLatih)
#raw_tf.to_csv("Hasil TF Data Latih.csv")
raw_tf

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
token,,,,,,,,,,,,,,,,,,,,,
a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acara,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ada,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,0,1,0
adab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adalah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
admin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Naive Bayes - Training (BoW)

In [24]:
def likelihood(term, dataLatih, dataUji):
    
    #tweet perkelas
    doc_positif = []
    doc_negatif = []
    for index, item in dataLatih.iterrows():
        if item['kelas'] == "positif":
            doc_positif.append(index+1)
        elif item['kelas'] == "negatif":
            doc_negatif.append(index+1)
    
    #total tf perkelas
    sum_positif = 0
    for item in doc_positif:
        sum_positif += term[str(item)].sum()
    
    sum_negatif = 0
    for item in doc_negatif:
        sum_negatif += term[str(item)].sum()
    
    
    #likelihood
    p_fitur_positif = []
    p_fitur_negatif = []
    for index, item in term.iterrows():
        count_term_positif = 0
        for i, d in enumerate(doc_positif):
            temp = term.at[str(index),str(doc_positif[i])]
            count_term_positif += temp
        p_fitur_positif.append((count_term_positif+1)/(sum_positif+len(term)))
        
        count_term_negatif = 0
        for i, d in enumerate(doc_negatif):
            temp = term.at[str(index),str(doc_negatif[i])]
            count_term_negatif += temp
        p_fitur_negatif.append((count_term_negatif+1)/(sum_negatif+len(term)))
    
    p_term = pd.DataFrame(term.index.values.tolist(), columns=['token'])
    p_term.set_index('token', inplace=True)
    p_term['P(fitur|positif)'] = p_fitur_positif
    p_term['P(fitur|negatif)'] = p_fitur_negatif
    
    
    #untuk data uji
    term_uji = []
    for index, item in dataUji.iterrows():
        term_uji += list(set(item['token']))
    
    term_unknown = []
    for item in term_uji:
        if item not in term.index.values.tolist():
            term_unknown.append(item)
    
    term_unique = np.unique(np.array(term_unknown))
    
    p_fitur_uji_positif = []
    p_fitur_uji_negatif = []
    for item in term_unique:
        p_fitur_uji_positif.append(1/(sum_positif+len(term)))
        p_fitur_uji_negatif.append(1/(sum_negatif+len(term)))
    
    p_term_uji = pd.DataFrame(term_unique, columns=['token'])
    p_term_uji.set_index('token', inplace=True)
    p_term_uji['P(fitur|positif)'] = p_fitur_uji_positif
    p_term_uji['P(fitur|negatif)'] = p_fitur_uji_negatif
    
    return p_term, p_term_uji

In [25]:
p_term_latih, p_term_uji = likelihood(term=raw_tf, dataLatih=cleanLatih, dataUji=cleanUji)

In [26]:
p_term_latih
#p_term_latih.to_csv("Hasil Likelihood Latih.csv")

,P(fitur|positif),P(fitur|negatif)
token,,
a,0.001607,0.000263
abis,0.000321,0.000790
ac,0.000643,0.000790
acara,0.000321,0.000527
ada,0.005464,0.007372
adab,0.000321,0.000527
adalah,0.000643,0.001053
adem,0.000964,0.000263
admin,0.000321,0.000790


In [27]:
p_term_uji

,P(fitur|positif),P(fitur|negatif)
token,,
abai,0.000321,0.000263
abang,0.000321,0.000263
amanah,0.000321,0.000263
bank,0.000321,0.000263
bayar,0.000321,0.000263
beginian,0.000321,0.000263
benerin,0.000321,0.000263
berkata,0.000321,0.000263
bodoh,0.000321,0.000263


# Naive Bayes - Testing (BoW)

In [28]:
def nb_bow(p_term_latih, p_term_uji, dataLatih, dataUji):
    
    #prior
    prior_positif = len(dataLatih.loc[dataLatih['kelas'] == 'positif'])/len(dataLatih)
    prior_negatif = len(dataLatih.loc[dataLatih['kelas'] == 'negatif'])/len(dataLatih)
        
    kelas_prediksi = []
    likelihood_positif = []
    likelihood_negatif = []
    for index, item in dataUji.iterrows():
        posterior_positif = prior_positif
        posterior_negatif = prior_negatif
        
        for item2 in list(item['token']):
            if item2 in p_term_latih.index:
                posterior_positif *= p_term_latih.at[item2, 'P(fitur|positif)']
            elif item2 in p_term_uji.index:
                posterior_positif *= p_term_uji.at[item2, 'P(fitur|positif)']
        
        for item2 in list(item['token']):
            if item2 in p_term_latih.index:
                posterior_negatif *= p_term_latih.at[item2, 'P(fitur|negatif)']
            elif item2 in p_term_uji.index:
                posterior_negatif *= p_term_uji.at[item2, 'P(fitur|negatif)']
        
        if posterior_positif > posterior_negatif:
            kelas_prediksi.append('positif')
        elif posterior_positif < posterior_negatif:
            kelas_prediksi.append('negatif')
        
        likelihood_positif.append(posterior_positif/prior_positif)
        likelihood_negatif.append(posterior_negatif/prior_negatif)
    
    dataUji['likelihood positif (bow)'] = likelihood_positif
    dataUji['likelihood negatif (bow)'] = likelihood_negatif
    dataUji['kelas prediksi (bow)'] = kelas_prediksi
    return dataUji

In [29]:
klasifikasi_bow = nb_bow(p_term_latih=p_term_latih, p_term_uji=p_term_uji, dataLatih=cleanLatih, dataUji=cleanUji)
klasifikasi_bow

,tweet,kelas aktual,normalisasi,clean,token,likelihood positif (bow),likelihood negatif (bow),kelas prediksi (bow)
0,Senangnya toilet stasiun MRT masih bersih! (@...,positif,Senang nya toilet stasiun MRT masih bersih!(@S...,senang nya toilet stasiun mrt masih bersih mrt...,"[senang, nya, toilet, stasiun, mrt, masih, ber...",4.448424e-36,2.738565e-37,positif
1,Biar ada pengalaman jalan2 k stasiun Lebak Bul...,positif,Biar ada pengalaman jalan-jalan ke stasiun Leb...,biar ada pengalaman jalan jalan ke stasiun leb...,"[biar, ada, pengalaman, jalan, jalan, ke, stas...",2.783279e-66,1.051908e-64,negatif
2,Cantik nih stasiun MRT nya @mrtjakarta,positif,Cantik ini stasiun MRT nya@mertjakarta,cantik ini stasiun mrt nya,"[cantik, ini, stasiun, mrt, nya]",5.347160e-12,4.270183e-12,positif
3,"Semoga rutenya cepat diperluas ya, nyaman bgt ...",positif,"Semoga rute nya cepat diperluas ya, nyaman ban...",semoga rute nya cepat diperluas ya nyaman banget,"[semoga, rute, nya, cepat, diperluas, ya, nyam...",6.804065e-21,1.349027e-23,positif
4,Hari ini pertama kali naik @mrtjakarta. Akhirn...,positif,Hari ini pertama kali naik@mertjakarta. Akhirn...,hari ini pertama kali naik akhirnya jakarta pu...,"[hari, ini, pertama, kali, naik, akhirnya, jak...",1.304195e-112,1.251274e-116,positif
5,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,positif,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,di stasiun mrt dukuh atas kondisi bersih dan n...,"[di, stasiun, mrt, dukuh, atas, kondisi, bersi...",5.320613e-44,2.321686e-42,negatif
6,Mewah dan Modern. 2 kata itu sepertinya cukup ...,positif,Mewah dan Modern. 2 kata itu sepertinya cukup ...,mewah dan modern kata itu sepertinya cukup men...,"[mewah, dan, modern, kata, itu, sepertinya, cu...",3.678796e-107,5.349375e-107,negatif
7,Salut dan makasi kpd mas penjaga di Stasiun Bl...,positif,Salut dan terima kasih kepada mas penjaga di S...,salut dan terima kasih kepada mas penjaga di s...,"[salut, dan, terima, kasih, kepada, mas, penja...",1.023676e-64,2.399816e-68,positif
8,"Mantap mrt jakarta!!! Keren, kalian sigap!",positif,"Mantap mrt jakarta!!! Keren, kalian sigap!",mantap mrt jakarta keren kalian sigap,"[mantap, mrt, jakarta, keren, kalian, sigap]",3.113282e-16,1.689189e-19,positif
9,gilak nih dingin nya gerbong.... top dah @mrtj...,positif,gila ini dingin nya gerbong\.. top sudah@mertj...,gila ini dingin nya gerbong top sudah,"[gila, ini, dingin, nya, gerbong, top, sudah]",4.143663e-20,3.795264e-20,positif


# Pelatihan POS Tagging

In [30]:
jumSample = 100000
namaFile = "tagged(4).txt"
with open(namaFile, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
 
pasangan = []
allPasangan = []
 
for line in lines[: min(jumSample, len(lines))]:
    if line == '':
        allPasangan.append(pasangan)
        pasangan = []
    else:
        kata, tag = line.split("\t")
        p = (kata,tag)
        pasangan.append(p)
 
ct = CRFTagger()
ct.train(allPasangan,'all_indo_man_tag_corpus_model.crf.tagger')

# Penentuan POS Tagging

In [31]:
def pos_tagging(corpus):
    list_tagged = []
    for tweet in corpus['normalisasi']:
        #datacleaning (link)
        tweet = re.sub(r'[a-zA-Z0-9]{0,}\.[c][o][m][^\s]{0,}', ' ', tweet)
        
        #datacleaning (username dan hashtag)
        tweet = re.sub(r'@\S*\s?|#\S*\s?', ' ', tweet) 
        
        #datacleaning (special character)
        tweet = re.sub(r'[\!\"\”\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\[\\\]\^\_\`\{\|\}\~]', ' ', tweet)
        
        hasil_tagged = ct.tag_sents([[item for item in word_tokenize(tweet)]])
        temp = pd.DataFrame(hasil_tagged[0], columns=['term', 'tag'])
        list_tagged.append(temp)
    
    return list_tagged

In [32]:
tagged_latih = pos_tagging(data_training)

# Hasil POS Tagging

In [33]:
tagged_uji = pos_tagging(data_testing)
tagged_uji

[        term  tag
 0     Senang   RB
 1        nya    X
 2     toilet  NNC
 3    stasiun  NNC
 4        MRT  NNP
 5      masih   RB
 6     bersih   JJ
 7        MRT  NNP
 8   Bundaran  NNP
 9         HI  NNP
 10        in  NNP
 11   Jakarta  NNP
 12       DKI  NNP
 13   Jakarta  NNP,              term  tag
 0            Biar   RP
 1             ada  VBT
 2      pengalaman  NNU
 3           jalan  NNU
 4           jalan  NNU
 5              ke   IN
 6         stasiun  NNP
 7           Lebak  NNP
 8           Bulus  NNP
 9             Dan  NNP
 10         selalu   RB
 11        membuat  VBT
 12             ku    X
 13      tersenyum    X
 14            MRT  NNP
 15          Lebak  NNP
 16          Bulus  NNP
 17         igshid    X
 18  1xv6dt7eh2ktu  NNU,       term  tag
 0   Cantik  NNP
 1      ini   DT
 2  stasiun  NNC
 3      MRT  NNP
 4      nya    X,         term  tag
 0     Semoga   SC
 1       rute  NNC
 2        nya    X
 3      cepat   JJ
 4  diperluas   JJ
 5         ya   RP


# Read Lexicon

In [34]:
with open('sentiwords_id (2).txt', 'r', encoding='utf-8') as f:
    sentiment_score = f.read().split('\n')

In [35]:
list_sentiment = []
for item in sentiment_score:
    kata, score = item.split(":")
    p = (kata,score)
    list_sentiment.append(p)

In [36]:
pd_sentiment = pd.DataFrame(list_sentiment, columns=['word','score'])
#pd_sentiment

In [37]:
with open('boosterwords_id.txt', 'r', encoding='utf-8') as f:
    booster_score = f.read().split('\n')

In [38]:
list_booster = []
for item in booster_score:
    kata, score = item.split(":")
    p = (kata,score)
    list_booster.append(p)

In [39]:
pd_booster = pd.DataFrame(list_booster, columns=['word','score'])
#pd_booster

# Lexicon-based Features

In [40]:
def lexicon_based(dataLatih, dataUji, sentimentCorpus, boosterCorpus):
    list_fitur = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']

    hasil = []
    
    for item in dataLatih:
        f1 = 0 #jumlah positif
        f2 = 0 #jumlah negatif
        f3 = 0 #jumlah JJ positif
        f4 = 0 #jumlah JJ negatif
        f5 = 0 #jumlah VB positif
        f6 = 0 #jumlah VB negatif
        f7 = 0 #jumlah RB positif
        f8 = 0 #jumlah RB negatif
        f9 = 0 #jumlah booster word
        
        for i, d in item.iterrows():
            terms = d['term'].lower()
            if terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) > 0:
                f1 += 1
            if terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) < 0:
                f2 += 1
        
        for i, d in item.iterrows():
            terms = d['term'].lower()
            if d['tag'] == 'JJ' and terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) > 0:
                f3 += 1
            if d['tag'] == 'JJ' and terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) < 0:
                f4 += 1
            if d['tag'] == 'VB' and terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) > 0:
                f5 += 1
            if d['tag'] == 'VB' and terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) < 0:
                f6 += 1
            if d['tag'] == 'RB' and terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) > 0:
                f7 += 1
            if d['tag'] == 'RB' and terms in sentimentCorpus['word'].tolist() and int(sentimentCorpus['score'][pd_sentiment['word'].tolist().index(terms)]) < 0:
                f8 += 1
        
        for i, d in item.iterrows():
            terms = d['term'].lower()
            if terms in boosterCorpus['word'].tolist():
                f9 += 1
        
        temp = [f1, f2, f3, f4, f5, f6, f7, f8, f9]
        hasil.append(temp)
    
    result = pd.DataFrame(list_fitur, columns=['fitur'])
    result.set_index('fitur', inplace=True)
    for i in range(len(dataLatih)):
        result[str(i+1)] = hasil[i]
    
    return result

In [41]:
lexiconBased = lexicon_based(dataLatih=tagged_latih, dataUji=tagged_uji, sentimentCorpus=pd_sentiment, boosterCorpus=pd_booster)
lexiconBased

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
fitur,,,,,,,,,,,,,,,,,,,,,
f1,7,2,1,3,0,0,0,0,0,0,...,0,1,1,0,1,3,1,2,0,0
f2,0,0,1,1,1,0,0,0,0,0,...,1,2,1,0,1,1,8,1,0,1
f3,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
f4,0,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,0,1
f5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
f9,0,0,0,0,1,0,0,0,0,0,...,0,2,1,0,1,2,2,0,0,0


In [42]:
lexiconBased2 = lexicon_based(dataLatih=tagged_uji, dataUji=tagged_uji, sentimentCorpus=pd_sentiment, boosterCorpus=pd_booster)
lexiconBased2

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
fitur,,,,,,,,,,,,,,,,,,,,,
f1,2,1,1,2,6,2,6,2,3,3,...,1,0,1,0,1,0,1,0,2,2
f2,0,0,0,0,1,2,0,1,0,0,...,5,1,3,1,1,3,2,1,3,3
f3,1,0,0,2,3,2,3,1,2,0,...,1,0,0,0,1,0,0,0,0,1
f4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,2,0,1,1,0
f5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f8,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
f9,0,0,0,1,2,2,0,1,0,0,...,3,0,2,0,0,0,1,0,1,1


# Naive Bayes - Training (Lexicon-based)

In [43]:
def likelihood_lb(datasetLatih, datasetUji, dataLatih):
    
    #tweet perkelas
    doc_positif = []
    doc_negatif = []
    for index, item in dataLatih.iterrows():
        if item['kelas'] == "positif":
            doc_positif.append(index+1)
        else:
            doc_negatif.append(index+1)
    
    #total tf perkelas
    sum_positif = 0
    for item in doc_positif:
        sum_positif += datasetLatih[str(item)].sum()
    
    sum_negatif = 0
    for item in doc_negatif:
        sum_negatif += datasetLatih[str(item)].sum()
    
    
    #likelihood
    p_fitur_positif = []
    p_fitur_negatif = []
    for index, item in datasetLatih.iterrows():
        count_fitur_positif = 0
        for i, d in enumerate(doc_positif):
            temp = datasetLatih.at[str(index),str(doc_positif[i])]
            count_fitur_positif += temp
        p_fitur_positif.append((count_fitur_positif+1)/(sum_positif+len(datasetLatih)))
        
        count_fitur_negatif = 0
        for i, d in enumerate(doc_negatif):
            temp = datasetLatih.at[str(index),str(doc_negatif[i])]
            count_fitur_negatif += temp
        p_fitur_negatif.append((count_fitur_negatif+1)/(sum_negatif+len(datasetLatih)))
    
    p_fitur = pd.DataFrame(datasetLatih.index.values.tolist(), columns=['fitur'])
    p_fitur.set_index('fitur', inplace=True)
    p_fitur['P(fitur|positif)'] = p_fitur_positif
    p_fitur['P(fitur|negatif)'] = p_fitur_negatif
    
    return p_fitur

In [44]:
peluang_fitur = likelihood_lb(datasetLatih=lexiconBased, datasetUji=lexiconBased2, dataLatih=cleanLatih)
peluang_fitur

,P(fitur|positif),P(fitur|negatif)
fitur,,
f1,0.477376,0.202643
f2,0.110860,0.361233
f3,0.226244,0.063877
f4,0.033937,0.127753
f5,0.002262,0.002203
f6,0.002262,0.002203
f7,0.015837,0.004405
f8,0.015837,0.035242
f9,0.115385,0.200441


# Naive Bayes - Testing (Lexicon-based)

In [45]:
def nb_lb(p_latih, datasetUji, dataUji, dataLatih):
    
    #prior
    prior_positif = len(dataLatih.loc[dataLatih['kelas'] == 'positif'])/len(data_latih)
    prior_negatif = len(dataLatih.loc[dataLatih['kelas'] == 'negatif'])/len(data_latih)
    
    kelas_prediksi = []
    likelihood_positif = []
    likelihood_negatif = []
    
    for index, item in dataUji.iterrows():
        posterior_positif = prior_positif
        posterior_negatif = prior_negatif
        
        for i, d in datasetUji.iterrows():
            posterior_positif *= math.pow(p_latih['P(fitur|positif)'][i], d[str(index+1)])
            posterior_negatif *= math.pow(p_latih['P(fitur|negatif)'][i], d[str(index+1)])
            
        if posterior_positif > posterior_negatif:
            kelas_prediksi.append('positif')
        else:
            kelas_prediksi.append('negatif')
        
        likelihood_positif.append(posterior_positif/prior_positif)
        likelihood_negatif.append(posterior_negatif/prior_negatif)
        
    dataUji['likelihood positif (lb)'] = likelihood_positif
    dataUji['likelihood negatif (lb)'] = likelihood_negatif
    dataUji['kelas prediksi (lb)'] = kelas_prediksi
    return dataUji

In [46]:
klasifikasi_lb = nb_lb(p_latih=peluang_fitur, datasetUji=lexiconBased2, dataUji=cleanUji, dataLatih=cleanLatih)
klasifikasi_lb

,tweet,kelas aktual,normalisasi,clean,token,likelihood positif (bow),likelihood negatif (bow),kelas prediksi (bow),likelihood positif (lb),likelihood negatif (lb),kelas prediksi (lb)
0,Senangnya toilet stasiun MRT masih bersih! (@...,positif,Senang nya toilet stasiun MRT masih bersih!(@S...,senang nya toilet stasiun mrt masih bersih mrt...,"[senang, nya, toilet, stasiun, mrt, masih, ber...",4.448424e-36,2.738565e-37,positif,8.165332e-04,1.155527e-05,positif
1,Biar ada pengalaman jalan2 k stasiun Lebak Bul...,positif,Biar ada pengalaman jalan-jalan ke stasiun Leb...,biar ada pengalaman jalan jalan ke stasiun leb...,"[biar, ada, pengalaman, jalan, jalan, ke, stas...",2.783279e-66,1.051908e-64,negatif,4.773756e-01,2.026432e-01,positif
2,Cantik nih stasiun MRT nya @mrtjakarta,positif,Cantik ini stasiun MRT nya@mertjakarta,cantik ini stasiun mrt nya,"[cantik, ini, stasiun, mrt, nya]",5.347160e-12,4.270183e-12,positif,4.773756e-01,2.026432e-01,positif
3,"Semoga rutenya cepat diperluas ya, nyaman bgt ...",positif,"Semoga rute nya cepat diperluas ya, nyaman ban...",semoga rute nya cepat diperluas ya nyaman banget,"[semoga, rute, nya, cepat, diperluas, ya, nyam...",6.804065e-21,1.349027e-23,positif,1.345934e-03,3.358410e-05,positif
4,Hari ini pertama kali naik @mrtjakarta. Akhirn...,positif,Hari ini pertama kali naik@mertjakarta. Akhirn...,hari ini pertama kali naik akhirnya jakarta pu...,"[hari, ini, pertama, kali, naik, akhirnya, jak...",1.304195e-112,1.251274e-116,positif,3.203610e-09,9.230846e-12,positif
5,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,positif,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,di stasiun mrt dukuh atas kondisi bersih dan n...,"[di, stasiun, mrt, dukuh, atas, kondisi, bersi...",5.320613e-44,2.321686e-42,negatif,3.022700e-08,3.095704e-08,negatif
6,Mewah dan Modern. 2 kata itu sepertinya cukup ...,positif,Mewah dan Modern. 2 kata itu sepertinya cukup ...,mewah dan modern kata itu sepertinya cukup men...,"[mewah, dan, modern, kata, itu, sepertinya, cu...",3.678796e-107,5.349375e-107,negatif,1.370548e-04,1.804755e-08,positif
7,Salut dan makasi kpd mas penjaga di Stasiun Bl...,positif,Salut dan terima kasih kepada mas penjaga di S...,salut dan terima kasih kepada mas penjaga di s...,"[salut, dan, terima, kasih, kepada, mas, penja...",1.023676e-64,2.399816e-68,positif,6.595076e-04,1.899239e-04,positif
8,"Mantap mrt jakarta!!! Keren, kalian sigap!",positif,"Mantap mrt jakarta!!! Keren, kalian sigap!",mantap mrt jakarta keren kalian sigap,"[mantap, mrt, jakarta, keren, kalian, sigap]",3.113282e-16,1.689189e-19,positif,5.568472e-03,3.395316e-05,positif
9,gilak nih dingin nya gerbong.... top dah @mrtj...,positif,gila ini dingin nya gerbong\.. top sudah@mertj...,gila ini dingin nya gerbong top sudah,"[gila, ini, dingin, nya, gerbong, top, sudah]",4.143663e-20,3.795264e-20,positif,1.087879e-01,8.321391e-03,positif


# Naive Bayes - Testing (BoW dan Lexicon-based)

In [47]:
def naive_bayes(dataLatih, hasil):
    
    #prior
    prior_positif = len(cleanLatih.loc[cleanLatih['kelas'] == 'positif'])/len(data_latih)
    prior_negatif = len(cleanLatih.loc[cleanLatih['kelas'] == 'negatif'])/len(data_latih)
    
    kelas_prediksi = []
    for index, item in hasil.iterrows():
        posterior_positif = prior_positif*hasil['likelihood positif (bow)'][index]*hasil['likelihood positif (lb)'][index]
        posterior_negatif = prior_negatif*hasil['likelihood negatif (bow)'][index]*hasil['likelihood negatif (lb)'][index]
        
        if posterior_positif > posterior_negatif:
            kelas_prediksi.append('positif')
        elif posterior_positif < posterior_negatif:
            kelas_prediksi.append('negatif')
    
    hasil['kelas prediksi fitur gabungan'] = kelas_prediksi
    return hasil

In [48]:
hasil_klasifikasi = naive_bayes(dataLatih=cleanLatih, hasil=klasifikasi_lb)
hasil_klasifikasi

,tweet,kelas aktual,normalisasi,clean,token,likelihood positif (bow),likelihood negatif (bow),kelas prediksi (bow),likelihood positif (lb),likelihood negatif (lb),kelas prediksi (lb),kelas prediksi fitur gabungan
0,Senangnya toilet stasiun MRT masih bersih! (@...,positif,Senang nya toilet stasiun MRT masih bersih!(@S...,senang nya toilet stasiun mrt masih bersih mrt...,"[senang, nya, toilet, stasiun, mrt, masih, ber...",4.448424e-36,2.738565e-37,positif,8.165332e-04,1.155527e-05,positif,positif
1,Biar ada pengalaman jalan2 k stasiun Lebak Bul...,positif,Biar ada pengalaman jalan-jalan ke stasiun Leb...,biar ada pengalaman jalan jalan ke stasiun leb...,"[biar, ada, pengalaman, jalan, jalan, ke, stas...",2.783279e-66,1.051908e-64,negatif,4.773756e-01,2.026432e-01,positif,negatif
2,Cantik nih stasiun MRT nya @mrtjakarta,positif,Cantik ini stasiun MRT nya@mertjakarta,cantik ini stasiun mrt nya,"[cantik, ini, stasiun, mrt, nya]",5.347160e-12,4.270183e-12,positif,4.773756e-01,2.026432e-01,positif,positif
3,"Semoga rutenya cepat diperluas ya, nyaman bgt ...",positif,"Semoga rute nya cepat diperluas ya, nyaman ban...",semoga rute nya cepat diperluas ya nyaman banget,"[semoga, rute, nya, cepat, diperluas, ya, nyam...",6.804065e-21,1.349027e-23,positif,1.345934e-03,3.358410e-05,positif,positif
4,Hari ini pertama kali naik @mrtjakarta. Akhirn...,positif,Hari ini pertama kali naik@mertjakarta. Akhirn...,hari ini pertama kali naik akhirnya jakarta pu...,"[hari, ini, pertama, kali, naik, akhirnya, jak...",1.304195e-112,1.251274e-116,positif,3.203610e-09,9.230846e-12,positif,positif
5,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,positif,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,di stasiun mrt dukuh atas kondisi bersih dan n...,"[di, stasiun, mrt, dukuh, atas, kondisi, bersi...",5.320613e-44,2.321686e-42,negatif,3.022700e-08,3.095704e-08,negatif,negatif
6,Mewah dan Modern. 2 kata itu sepertinya cukup ...,positif,Mewah dan Modern. 2 kata itu sepertinya cukup ...,mewah dan modern kata itu sepertinya cukup men...,"[mewah, dan, modern, kata, itu, sepertinya, cu...",3.678796e-107,5.349375e-107,negatif,1.370548e-04,1.804755e-08,positif,positif
7,Salut dan makasi kpd mas penjaga di Stasiun Bl...,positif,Salut dan terima kasih kepada mas penjaga di S...,salut dan terima kasih kepada mas penjaga di s...,"[salut, dan, terima, kasih, kepada, mas, penja...",1.023676e-64,2.399816e-68,positif,6.595076e-04,1.899239e-04,positif,positif
8,"Mantap mrt jakarta!!! Keren, kalian sigap!",positif,"Mantap mrt jakarta!!! Keren, kalian sigap!",mantap mrt jakarta keren kalian sigap,"[mantap, mrt, jakarta, keren, kalian, sigap]",3.113282e-16,1.689189e-19,positif,5.568472e-03,3.395316e-05,positif,positif
9,gilak nih dingin nya gerbong.... top dah @mrtj...,positif,gila ini dingin nya gerbong\.. top sudah@mertj...,gila ini dingin nya gerbong top sudah,"[gila, ini, dingin, nya, gerbong, top, sudah]",4.143663e-20,3.795264e-20,positif,1.087879e-01,8.321391e-03,positif,positif


# Deteksi Target

In [49]:
import nltk
import sklearn
import pycrfsuite

# Chunking

In [50]:
#chunking = tagged_uji.copy()

In [51]:
grammar = ('''
    NP: {<NNP|NNC>+<CC|IN><NNP|NNC>+}
    NP: {<NNP|NNC>+<JJ>}
    NP: {<NNP|NNC>+<CD\w+>}
    NP: {<NNP|NNC>+<NNP|NNC>+}
    NP: {<NNP|NNC>}
    NP: {<FW>+}
    ''')

#NP: {<NNP|NNC>+<DT>}
#NP: {<FW>+}
#NP: {<NN>+<NNP>+}
#NP: {<NNP>+<NN>+}
#NP: {<NNP>+}
#NP: {<NNG>}
#NP: {<NP><NP>+}
#NP: {<PRP><CC><NN?\w+>}
#NP: {<NN?\w+><CC><PRP>}
#NP: {<NN?\w+>+<PRP>}
#NP: {<CD\w+>*<NN?\w+>+<CD\w+>}
#NP: {<CD\w+><NN?\w+>+<CD\w+>*}

In [52]:
chunkParser = nltk.RegexpParser(grammar)

In [53]:
from operator import itemgetter

In [54]:
def target_detection(chunking, dataUji):
    objek = []
    for df in chunking:
        objek_tweet = []
        tweet_uji = [tuple(x) for x in df.to_records(index=False)]
        tree = chunkParser.parse(tweet_uji)
        #tree.draw()
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
            list_tuple = subtree.leaves()
            res = list(map(itemgetter(0), list_tuple))
            objek_tweet.append(res)
        objek.append(objek_tweet)
    
    dataUji['objek opini'] = objek
    return dataUji

In [55]:
target_objek = target_detection(chunking=tagged_uji, dataUji=cleanUji)
objek = target_objek.copy()
objek.drop(columns= ['normalisasi', 'clean', 'token', 'likelihood positif (bow)', 'likelihood negatif (bow)', 'kelas prediksi (bow)', 'likelihood positif (lb)', 'likelihood negatif (lb)', 'kelas prediksi (lb)'])

,tweet,kelas aktual,kelas prediksi fitur gabungan,objek opini
0,Senangnya toilet stasiun MRT masih bersih! (@...,positif,positif,"[[toilet, stasiun, MRT], [MRT, Bundaran, HI, i..."
1,Biar ada pengalaman jalan2 k stasiun Lebak Bul...,positif,negatif,"[[stasiun, Lebak, Bulus, Dan], [MRT, Lebak, Bu..."
2,Cantik nih stasiun MRT nya @mrtjakarta,positif,positif,"[[Cantik], [stasiun, MRT]]"
3,"Semoga rutenya cepat diperluas ya, nyaman bgt ...",positif,positif,[[rute]]
4,Hari ini pertama kali naik @mrtjakarta. Akhirn...,positif,positif,"[[Jakarta], [fasilitas], [stasiun], [MRT], [na..."
5,Di stasiun MRT Dukuh Atas. Kondisi bersih dan ...,positif,negatif,"[[stasiun, MRT, Dukuh, Atas, Kondisi, bersih],..."
6,Mewah dan Modern. 2 kata itu sepertinya cukup ...,positif,positif,"[[kata], [MRT, Jakarta, sejauh], [Stasiun], [A..."
7,Salut dan makasi kpd mas penjaga di Stasiun Bl...,positif,positif,"[[Salut], [mas, penjaga, di, Stasiun, Blok, M]..."
8,"Mantap mrt jakarta!!! Keren, kalian sigap!",positif,positif,"[[Mantap, mrt, jakarta, Keren]]"
9,gilak nih dingin nya gerbong.... top dah @mrtj...,positif,positif,[[gila]]


In [56]:
#objek.to_csv("Hasil Analisis Sentimen.csv")